In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import confusion_matrix,recall_score,classification_report , accuracy_score , average_precision_score, f1_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
import eli5 
from eli5.sklearn import PermutationImportance
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import export_graphviz
from IPython.display import Image
import pydot #pip install pydot
import xgboost as xgb
from xgboost import XGBClassifier
import itertools
from sklearn.pipeline import Pipeline, make_pipeline
from scipy.stats import skew
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Lasso
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.ensemble import StackingClassifier, StackingRegressor

ModuleNotFoundError: No module named 'eli5'

In [3]:
best_c = printing_Kfold_scores(X_train,y_train)

NameError: name 'X_train' is not defined

# 評分

In [4]:
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    绘制混淆矩阵
    """
#     plt.figure(figsize = (50,50))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

NameError: name 'plt' is not defined

In [1]:
def Regression_eva_plot(y_pred, y_test):
    figsize(8, 8)
    # Density plot of the final predictions and the test values
    sns.kdeplot(y_pred, label = 'Predictions')
    sns.kdeplot(y_test, label = 'Values')

    # Label the plot
    plt.xlabel('Target'); plt.ylabel('Density');
    plt.title('Test Values and Predictions');

In [2]:
def Regression_rmse_cv(model,X_test,y_test,class_names):
    y_pred = model.predict(X_test)
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=5))
    print("mean rmse: ",np.mean(rmse) )
    Regression_eva_plot(y_pred, y_test)
    return y_pred

In [3]:
def Regression_error_displot(y_pred, y_test):
    figsize = (6, 6)

    # Calculate the residuals 
    residuals = y_pred - y_test

    # Plot the residuals in a histogram
    plt.hist(residuals, color = 'red', bins = 20,
             edgecolor = 'black')
    plt.xlabel('Error'); plt.ylabel('Count')
    plt.title('Distribution of Residuals');

In [5]:
def Classify_evaluate(model, X_test,y_test, class_names):
    y_pred = model.predict(X_test)
    cnf_matrix = confusion_matrix(y_test,y_pred)
    np.set_printoptions(precision=2)

    print("召回率: ", recall_score(y_test.values, y_pred,average='macro') ) #多分類用macro   二分類去掉
    print("ACC: ", accuracy_score(y_test.values, y_pred) )
#     print("精確: ", average_precision_score(y_test.values, y_pred,average='macro') )
    print("F1: ", f1_score(y_test.values, y_pred,average='macro') )
#     print("average_precision_score: ", average_precision_score(y_test.values, y_pred))  muticlass is not supported
    # 绘制
    class_names = class_names
    plt.figure()
    plot_confusion_matrix(cnf_matrix
                          , classes=class_names
                          , title='Confusion matrix')
    plt.show()
    return y_pred

# 模型檢視

In [3]:
perm = PermutationImportance(lr, random_state=1).fit(X_test, y_test.values.reshape(-1, 1))
eli5.show_weights(perm, feature_names = X_test.columns.tolist())

NameError: name 'PermutationImportance' is not defined

In [2]:
def Importance_feature(model,feature_list):
    importances = list(model.feature_importances_)

# 名字，数值组合在一起
    feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

    # 排序
    feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

    # 打印出来 
    [print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];
    plt.style.use('fivethirtyeight')
#     plt.figure(figsize=(5*len(importances),30))
    # 指定位置
    x_values = list(range(len(importances)))

    # 绘图
    plt.bar(x_values, importances, orientation = 'vertical', color = 'r', edgecolor = 'k', linewidth = 1.2)

    # x轴名字得竖着写
    plt.xticks(x_values, feature_list, rotation='vertical')

    # 图名
    plt.ylabel('Importance'); plt.xlabel('Variable'); plt.title('Variable Importances');

In [ ]:
Importance_feature(best_grid_rf, X_train.columns)

## 樹可視化

In [ ]:
# 拿到其中的一棵树
tree = best_grid_rf2.estimators_[5]

# 导出成dot文件
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)

# 绘图
(graph, ) = pydot.graph_from_dot_file('tree.dot')

# 展示
Image(graph.create_png())

In [ ]:
def print_graph(clf, feature_names, class_name):
    """Print decision tree."""
    graph = export_graphviz(
        clf,
        label="root",
        proportion=True,
        impurity=False, 
        out_file=None, 
        feature_names=feature_names,
        class_names=class_name,
        filled=True,
        rounded=True,
        precision = 1
    )
    (graph, ) = pydot.graph_from_dot_data(graph)
#     graph = pydotplus.graph_from_dot_data(graph)  
    return Image(graph.create_png())

In [ ]:
print_graph(tree, feature_list, {0:"dead", 1:"lives"})

# Lr

In [ ]:
def printing_Kfold_scores(X,Y):
    y_train_data =Y.copy()
    x_train_data = X.copy()
    
    y_train_data.reset_index(drop = True, inplace = True)
    x_train_data.reset_index(drop = True, inplace = True)
    print(x_train_data)
    print(y_train_data)
    fold = KFold(5,shuffle=False) 

    # 定义不同力度的正则化惩罚力度
    c_param_range = [0.01,0.1,1,10,100]
    # 展示结果用的表格
    results_table = pd.DataFrame(index = range(len(c_param_range),2), columns = ['C_parameter','Mean recall score'])
    results_table['C_parameter'] = c_param_range

    # k-fold 表示K折的交叉验证，这里会得到两个索引集合: 训练集 = indices[0], 验证集 = indices[1]
    j = 0
    #循环遍历不同的参数
    for c_param in c_param_range:
        print('-------------------------------------------')
        print('正则化惩罚力度: ', c_param)
        print('-------------------------------------------')
        print('')

        recall_accs = []
        
        #一步步分解来执行交叉验证
        for iteration, (train_index, test_index) in enumerate(fold.split(x_train_data),start=1):   #indice[0] = train_index , indices[1] = test_index 

            # 指定算法模型，并且给定参数
#             lr = LogisticRegression(solver = "liblinear",C = c_param, penalty = 'l1')
            lr = LogisticRegression(solver = "newton-cg",C = c_param, penalty = 'l2')
#             lr = LogisticRegression(solver = "liblinear",C = c_param, penalty = 'l1', random_state=0)
            # 训练模型，注意索引不要给错了，训练的时候一定传入的是训练集，所以X和Y的索引都是0
#             print(train_index)
#             print(test_index)
            X_train, X_test = x_train_data.loc[train_index,:], x_train_data.loc[test_index,:]
            y_train, y_test = y_train_data.loc[train_index], y_train_data.loc[test_index]
#             print(y_train)
            lr.fit(X_train,y_train.values.ravel())

            # 建立好模型后，预测模型结果，这里用的就是验证集，索引为1
            y_pred = lr.predict(X_test.values)

            # 有了预测结果之后就可以来进行评估了，这里recall_score需要传入预测值和真实值。
            recall_acc = recall_score(y_test.values,y_pred, average='macro')   #多分類用macro   二分類去掉
            # 一会还要算平均，所以把每一步的结果都先保存起来。
            recall_accs.append(recall_acc)
            print('Iteration ', iteration,': 召回率 = ', recall_acc)

        # 当执行完所有的交叉验证后，计算平均结果
        results_table.loc[j,'Mean recall score'] = np.mean(recall_accs)
        j += 1
        print('')
        print('平均召回率 ', np.mean(recall_accs))
        print('')
#         print(results_table)
    #找到最好的参数，哪一个Recall高，自然就是最好的了。
    best_c = results_table.loc[results_table['Mean recall score'].astype('float32').idxmax()]['C_parameter']
    
    # 打印最好的结果
    print('*********************************************************************************')
    print('效果最好的模型所选参数 = ', best_c)
    print('*********************************************************************************')
    
    return best_c

In [ ]:
best_c = printing_Kfold_scores(X_train,y_train)

In [ ]:
lr = LogisticRegression(solver = "newton-cg",C = best_c, penalty = 'l2')
lr.fit(X_train,y_train.values.ravel())
y_pred = Classify_evaluate(lr, X_test,y_test, lb_Y.classes_)

In [ ]:
params={"solver" : ["newton-cg"],"C":np.logspace(-3,3,7), "penalty":["l2"]}# l1 lasso l2 ridge
lr=LogisticRegression()
grid_lr=GridSearchCV(lr,params,cv=10)
grid_lr.fit(X_train,y_train)

In [ ]:
best_lr = grid_lr.best_estimator_
y_pred = Classify_evaluate(best_lr, X_test,y_test, lb_Y.classes_)

# SVM

In [7]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
#scores = ['precision', 'recall']

grid_svm = GridSearchCV(
    SVC(), tuned_parameters, scoring='f1_macro')
grid_svm.fit(X_train, y_train)
grid_svm.best_params_

NameError: name 'GridSearchCV' is not defined

In [ ]:
best_svm = grid_svm.best_estimator_
y_pred = Classify_evaluate(best_svm, X_test,y_test, lb_Y.classes_)

# Bay

In [ ]:
Nb = GaussianNB()  
Nb.fit(X_train, y_train)

In [ ]:
y_pred = Classify_evaluate(Nb, X_test,y_test, lb_Y.classes_)

# RandomRF

In [ ]:
# 建立树的个数
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# 最大特征的选择方式
max_features = ['auto', 'sqrt']
# 树的最大深度
max_depth = [int(x) for x in np.linspace(10, 20, num = 2)]
max_depth.append(None)
# 节点最小分裂所需样本个数
min_samples_split = [2, 5, 10]
# 叶子节点最小样本数，任何分裂不能让其子节点样本数少于此值
min_samples_leaf = [1, 2, 4]
# 样本采样方法
bootstrap = [True, False]

# Random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

#訓練樹
# 随机选择最合适的参数组合
rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,
                              n_iter = 100, scoring='recall', 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1)

# 执行寻找操作
rf_random.fit(X_train, y_train)
rf_random.best_params_

In [ ]:
best_random = rf_random.best_estimator_
y_pred = Classify_evaluate(best_random, X_test,y_test, lb_Y.classes_)

## GridRF

In [ ]:
# 网络搜索
# param_grid = {
#     'bootstrap': [True],
#     'max_depth': [8,10,12],
#     'max_features': ['auto'],
#     'min_samples_leaf': [2,3, 4, 5,6],
#     'min_samples_split': [3, 5, 7],
#     'n_estimators': [800, 900, 1000, 1200]
# }
param_grid2 = {
    'bootstrap': [False],
    'min_samples_leaf': [2],
    'max_features': ['auto'],
    'max_depth': [10],
    'min_samples_split': [17],
    'n_estimators': [400]
}
# 选择基本算法模型
rf = RandomForestClassifier()

# 网络搜索
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid2, 
                           scoring = 'f1', cv = 3, 
                           n_jobs = -1, verbose = 2)
grid_search.fit(X_train,y_train)
grid_search.best_params_

In [ ]:
best_grid_rf = grid_search.best_estimator_
Classify_evaluate(best_grid_rf,X_test,y_test, class_names = label_le.classes_)

## LightGBM

In [ ]:
# parameters = {
#     "max_depth":[10,15],
#     "learning_rate": [0.01,0.02],
#     "feature_fraction":[0.9,1.2,1.5],
#     "bagging_fraction" : [0.3,0.5,0.6],
#     "bagging_freq": [2,3,4],
#     "lambda_l1" : [0,0.1],
#     "lambda_l2" : [0,5,7,10],
#     "cat_smooth" : [5,7,10],
#     "n_estimators" :[100000, 150000],
#     "objective" : ['binary'],
#     "class_weight" : ['balanced'],
#     "reg_alpha" : [0.1], 
#      "reg_lambda" : [0.1],
#     "subsample" : [0.8],
#      "n_jobs" : [-1], "random_state" : [50]
# }

import lightgbm as lgb

lgb_model = lgb.LGBMClassifier()
parameters = {
    "n_estimators" : [10000],
    'max_depth': [15],
    'bagging_fraction': [0.9],
    'bagging_freq': [6],
    "feature_fraction":[0.9],
    "objective" : ['binary'],
    "class_weight" : ['balanced'],
    'cat_smooth': [5],
    'feature_fraction': [0.9],
    'lambda_l1': [0.1],
    'lambda_l2': [7],
    'learning_rate': [0.05],
    "reg_alpha" : [0.1], 
     "reg_lambda" : [0.1],
    "subsample" : [0.8],
     "n_jobs" : [-1], "random_state" : [50]
    }
lgb_model_search = GridSearchCV(lgb_model, parameters, cv =3, scoring = 'accuracy', verbose = 2, n_jobs = -1)
lgb_model_search.fit(X_train, y_train)
lgb_model_search.best_params_

In [ ]:
best_lgb = lgb_model_search.best_estimator_
y_pred = Classify_evaluate(best_lgb, X_test,y_test, lb_Y.classes_)

## xgboost

In [ ]:
xgb_model = xgb.XGBClassifier()

#brute force scan for all parameters, here are the tricks
#usually max_depth is 6,7,8
#learning rate is around 0.05, but small changes may make big diff
#tuning min_child_weight subsample colsample_bytree can have 
#much fun of fighting against overfit 
#n_estimators is how many round of boosting
#finally, ensemble xgboost with multiple seeds may reduce variance
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['binary:logistic'],
              'learning_rate': [0.05], #so called `eta` value
              'max_depth': [8],   #max_depth [default=6] range: [0,∞] (0 is only accepted in lossguided growing policy when tree_method is set as hist)
              'min_child_weight': [6], # [default=1] range: [0,∞]
              "gamma" : [0.4] , #default=0, range: [0,∞]  降低你和風險
              'subsample': [0.5],  # [default=1] range: (0,1]
              'colsample_bytree': [0.5],
              'n_estimators': [5000], #number of trees, change it to 1000 for better results
              "booster":['gbtree'],
              'seed': [1337]}


xgb_grid = GridSearchCV(xgb_model, parameters, n_jobs=5, 
                   cv=5,   
                   scoring='f1_macro',    #muticlass use f1_macro ,other use acc
                   verbose=2, refit=True)
xgb_grid.fit(X_train,y_train)
xgb_grid.best_params

In [ ]:
best_xgb = xgb_grid.best_estimator_
y_pred = Classify_evaluate(best_xgb, X_test,y_test, lb_Y.classes_)

## Gradient

In [ ]:
trees_grid = {'n_estimators': [100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800]}

model = GradientBoostingRegressor(loss = 'lad', max_depth = 5,
                                  min_samples_leaf = 6,
                                  min_samples_split = 6,
                                  max_features = None,
                                  random_state = 42)

# Grid Search Object using the trees range and the random forest model
grad_search = GridSearchCV(estimator = model, param_grid=trees_grid, cv = 4, 
                           scoring = 'neg_mean_absolute_error', verbose = 1,
                           n_jobs = -1, return_train_score = True)
grid_search.fit(X, train_labels)
grid_search.best_params

In [ ]:
best_grad = grad_search.best_estimator_
y_pred = Classify_evaluate(best_xgb, X_test,y_test, lb_Y.classes_)


## Stacking

In [6]:
class stacking(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self,mod,meta_model):
        self.mod = mod
        self.meta_model = meta_model
        self.kf = KFold(n_splits=5, random_state=42, shuffle=True)
        
    def fit(self,X,y):
        self.saved_model = [list() for i in self.mod]
        oof_train = np.zeros((X.shape[0], len(self.mod)))
        
        for i,model in enumerate(self.mod):
            for train_index, val_index in self.kf.split(X,y):
                renew_model = clone(model)
                renew_model.fit(X[train_index], y[train_index])
                self.saved_model[i].append(renew_model)
                oof_train[val_index,i] = renew_model.predict(X[val_index])
        
        self.meta_model.fit(oof_train,y)
        return self
    
    def predict(self,X):
        whole_test = np.column_stack([np.column_stack(model.predict(X) for model in single_model).mean(axis=1) 
                                      for single_model in self.saved_model]) 
        return self.meta_model.predict(whole_test)
    
    def get_oof(self,X,y,test_X):
        oof = np.zeros((X.shape[0],len(self.mod)))
        test_single = np.zeros((test_X.shape[0],5))
        test_mean = np.zeros((test_X.shape[0],len(self.mod)))
        for i,model in enumerate(self.mod):
            for j, (train_index,val_index) in enumerate(self.kf.split(X,y)):
                clone_model = clone(model)
                clone_model.fit(X[train_index],y[train_index])
                oof[val_index,i] = clone_model.predict(X[val_index])
                test_single[:,j] = clone_model.predict(test_X)
            test_mean[:,i] = test_single.mean(axis=1)
        return oof, test_mean

NameError: name 'BaseEstimator' is not defined

In [ ]:
"""
Stacking regression models
"""

In [ ]:
def get_models():
    models = dict()
    models['lr'] = LogisticRegression()
    models['Rf'] = RandomForestClassifier()
    models['xgb'] = XGBClassifier()
    models['svm'] = SVC()
    models['bayes'] = GaussianNB()
    models['lgb'] = lgb.LGBMClassifier()
    models['stacking'] = get_stacking()
    return models

In [ ]:
def get_base_stacking():
    # define the base models
    level0 = list()
    level0.append(('lr', LogisticRegression()))
    level0.append(('Rf',RandomForestClassifier()))
    level0.append(('xgb', XGBClassifier()))
    level0.append(('svm', SVC()))
    level0.append(('bayes', GaussianNB()))
#     level0.append(('lgb', lgb.LGBMClassifier()))
    # define meta learner model
    level1 = lgb.LGBMClassifier()
    # define the stacking ensemble
    model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
    return model

In [ ]:
# get a stacking ensemble of models
def get_best_stacking():
    # define the base models
    level0 = list()
    level0.append(('lr', lr))
    level0.append(('Rf', best_random))
    level0.append(('xgb', best_xgb))
    level0.append(('svm', best_svm))
    level0.append(('bayes', Nb))
    level0.append(('lgb', best_lgb))
    # define meta learner model
    level1 = LogisticRegression()
    # define the stacking ensemble
    model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
    return model

In [ ]:
def evaluate_model(model, X, y):
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	scores = cross_val_score(model, X, y, scoring='f1_macro', cv=cv, n_jobs=-1, error_score='raise')
	return scores

In [ ]:
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_model(model, X_train, y_train)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, np.mean(scores), np.std(scores)))
# plot model performance for comparison
plt.boxplot(results, labels=names, showmeans=True)
plt.show()

In [ ]:
model = get_best_stacking()

In [ ]:
y_pred = Classify_evaluate(model, X_test,y_test, lb_Y.classes_)

## 回歸stacking

In [ ]:
def get_models():
    models = dict()
    models['lr'] = LinearRegression()
    models['Rf'] = RandomForestRegressor()
    models['xgb'] = XGBRegressor()
    models['svm'] = SVR(C = 1000, gamma = 0.1)
    models['bayes'] = BayesianRidge()
    models['lgb'] = lgb.LGBMRegressor()
    models['stacking'] = get_base_stacking()
    return models

In [ ]:
def get_base_stacking():
    # define the base models
    level0 = list()
    level0.append(('lr', LinearRegression()))
    level0.append(('Rf',RandomForestRegressor()))
    level0.append(('xgb', XGBRegressor()))
    level0.append(('svm', SVR(C = 1000, gamma = 0.1)))
    level0.append(('bayes', BayesianRidge()))
    level0.append(('lgb', lgb.LGBMRegressor()))
    # define meta learner model
    level1 = lgb.LGBMRegressor()
    # define the stacking ensemble
    model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)
    return model

In [ ]:
def evaluate_model(model, X, y):
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
	return scores

In [ ]:
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_model(model, X, y)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, np.mean(scores), np.std(scores)))
# plot model performance for comparison
plt.boxplot(results, labels=names, showmeans=True)
plt.show()